# 1. Учим conv сеть для классификации - выбить auc выше 0.95

In [12]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [1]:
import pandas as pd

#df_train = pd.read_csv("data/train.csv")
#df_test = pd.read_csv("data/test.csv")
#df_val = pd.read_csv("data/val.csv")

df = pd.read_excel("отзывы за лето.xls")
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


### Предобработка

In [2]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [3]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df['text'] = df['Content'].apply(preprocess_text)
df.head()

,Rating,Content,Date,text
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро


In [7]:
train_corpus = " ".join(df["text"])
#train_corpus = train_corpus.lower()

In [9]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\a.kraev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [10]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [13]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [15]:
tokens_filtered_top[:20]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер',
 'быстро',
 'обновление',
 'мочь',
 'пароль',
 'антивирус',
 'пользоваться',
 'банк',
 'сбербанк',
 'устраивать',
 'вход']

In [27]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.3,  random_state=42)
df_train, df_val  = train_test_split(df_train, test_size=0.3, random_state=42)

In [29]:
import numpy as np

vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

import keras

num_classes = 5
y_train = keras.utils.to_categorical(df_train["class"], num_classes)
y_val = keras.utils.to_categorical(df_val["class"], num_classes)

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [24]:
x_train.shape

(10122, 40)

In [25]:
max_len

40

In [26]:
x_train[1]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0, 19, 14, 32, 21, 51])

# Keras model

In [24]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [23]:
model = Sequential()

model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))



model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Dropout(0.1))

model.add(Conv1D(128, 3))
model.add(Activation("elu"))
model.add(Dropout(0.3))

model.add(Conv1D(128, 3))
model.add(Activation("elu"))
model.add(Dropout(0.1))

model.add(Conv1D(128, 3))
model.add(Activation("elu"))
model.add(Dropout(0.1))

# model.add(Conv1D(128, 3))
# model.add(Activation("elu"))
# model.add(Dropout(0.1))

# model.add(Conv1D(128, 3))
# model.add(Activation("elu"))
# model.add(Dropout(0.1))

model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))



In [24]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy','AUC'])

In [25]:
early_stopping=EarlyStopping(monitor='val_loss',
                             patience=10,
                            mode="min")  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[early_stopping])

Epoch 1/20
  1/319 [..............................] - ETA: 0s - loss: 0.6953 - accuracy: 0.4551WARNING:tensorflow:From /home/rzaharov@mvs.local/.local/share/virtualenvs/nlp-course-A3nIyBIP/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
319/319 [==============================] - 69s 215ms/step - loss: 0.6285 - accuracy: 0.6150 - val_loss: 0.6169 - val_accuracy: 0.6219
Epoch 2/20
319/319 [==============================] - 15s 46ms/step - loss: 0.6159 - accuracy: 0.6259 - val_loss: 0.6148 - val_accuracy: 0.6289
Epoch 3/20
319/319 [==============================] - 15s 48ms/step - loss: 0.6121 - accuracy: 0.6304 - val_loss: 0.6129 - val_accuracy: 0.6301
Epoch 4/20
319/319 [==============================] - 13s 42ms/step - loss: 0.6078 - accuracy: 0.6331 - val_loss: 0.6129 - val_accuracy: 0.6232
Ep

In [30]:
score = model.evaluate(x_val, y_val, batch_size=batch_size)
print('Val score:', score[0])
print('Val AUC:', score[1])

NameError: name 'model' is not defined

In [27]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

45/45 [==============================] - 0s 8ms/step


2. Предобучаем word2vec и его эмбединга инициализируем сетку, как влияет на качество?

In [31]:
from gensim.models import Word2Vec

In [32]:
df.head()

,Rating,Content,Date,text
0,5,It just works!,2017-08-14,it just works
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,целое удобноной приложениеиз минус хотеть боль...
2,5,Отлично все,2017-08-14,отлично
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,зависать 1 работа антивирус ранее пользоваться...
4,5,"Очень удобно, работает быстро.",2017-08-14,удобно работать быстро


In [43]:
sentences = [[i for i in document.split()] for document in df['text'].tolist()]

In [44]:
model_w2v = Word2Vec(sentences=sentences, 
                     size=200, 
                     window=5,
                     min_count=2, 
                     sg=1, 
                     hs=0, 
                     negative=10, 
                     workers=32, 
                     seed=34)

model_w2v.train(sentences, total_examples=len(sentences), epochs=20)

(1486248, 2062320)

In [46]:
model_w2v.most_similar(positive=['минус'])

<ipython-input-46-6fd74d567873>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model_w2v.most_similar(positive=['минус'])


[('единственный', 0.5451143980026245),
 ('существенный', 0.5310849547386169),
 ('упасть', 0.531042754650116),
 ('жирный', 0.5271909236907959),
 ('плюс', 0.5227070450782776),
 ('тратиться', 0.5189428329467773),
 ('одинна', 0.51517653465271),
 ('аванс', 0.5119965672492981),
 ('какнуть', 0.5105953216552734),
 ('доставляетнеудобство', 0.5087352991104126)]

In [47]:
model = Sequential()

model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len, input_length=max_len, weights=[modelW2V.trainables.syn1neg[:max_words]]))

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

NameError: name 'Sequential' is not defined

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy','AUC'])

In [ ]:
early_stopping=EarlyStopping(monitor='val_loss',
                             patience=10,
                            mode="min")  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[early_stopping])